In [5]:
import pandas as pd
import numpy as np
from catboost import Pool, CatBoostRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import TimeSeriesSplit 
import optuna 
import json
from sklearn.metrics import mean_absolute_error

In [6]:
#Load inn datasets
X_test  = pd.read_parquet('../data/prepared_datasets/avg/no_duplicates/X_test.parquet')
X_train = pd.read_parquet('../data/prepared_datasets/avg/no_duplicates/X_train.parquet')
y_train = pd.read_parquet('../data/prepared_datasets/avg/no_duplicates/Y_train.parquet')
y_train_a = pd.read_parquet('../data/prepared_datasets/avg/no_duplicates/Y_train_a.parquet')
y_train_b = pd.read_parquet('../data/prepared_datasets/avg/no_duplicates/Y_train_b.parquet')
y_train_c = pd.read_parquet('../data/prepared_datasets/avg/no_duplicates/Y_train_c.parquet')

In [7]:
def test_train_split(df):
    dates_2 = (df.index >= '2023-04-01') & (df.index <= '2023-04-15')
    dates_1 = (df.index >= '2021-05-01') & (df.index <= '2021-08-01')

    test_set = df[dates_1 | dates_2]

    training_set = df[~(dates_1 | dates_2)]

    X_train = training_set.drop("pv_measurement", axis=1)
    y_train = training_set['pv_measurement']

    X_test = test_set.drop("pv_measurement", axis=1)
    y_test = test_set['pv_measurement'] 

    
    
    return X_train, X_test, y_train, y_test


X_train_new_c, X_test_new_c, y_train_new_c, y_test_c = test_train_split(pd.concat([X_train[X_train["location"] == "C"].drop("location", axis=1), y_train_c], axis=1))

In [8]:
#Create a pool of data

train_pool_c = Pool(X_train_new_c, y_train_new_c)
test_pool_c = Pool(X_test_new_c) 

In [9]:
#For location A
def objective(trial, X_train, y_train):
    params = {
        "iterations": trial.suggest_int("iterations", 300, 3500),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 13),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
        "l2_leaf_reg": trial.suggest_int("l2_leaf_reg", 2, 10),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.3, 1.0),
        "random_strength": trial.suggest_float("random_strength", 0.3, 1.0),
        "border_count": trial.suggest_int("border_count", 1, 1000),
        "rsm": trial.suggest_float("rsm", 0.05, 1),
        "loss_function" :"LogCosh"
    }

    catboost_model_c = CatBoostRegressor(**params, verbose=400)
    catboost_model_c.fit(train_pool_c)
    pred_c = pd.DataFrame(catboost_model_c.predict(test_pool_c))
    MAE_c = mean_absolute_error(y_test_c, pred_c)
    return MAE_c

study = optuna.create_study(direction='minimize')
study.optimize(lambda trial: objective(trial, X_train, y_train), n_trials=80)

[I 2023-11-11 12:00:38,670] A new study created in memory with name: no-name-552070e4-534a-4372-92d2-10ef55f54240


0:	learn: 63.4097858	total: 60.5ms	remaining: 2m 36s
400:	learn: 37.5424608	total: 5.25s	remaining: 28.8s
800:	learn: 26.1922979	total: 10.7s	remaining: 24s
1200:	learn: 22.0917985	total: 16.2s	remaining: 18.9s
1600:	learn: 20.5802483	total: 21.8s	remaining: 13.6s
2000:	learn: 19.9185397	total: 27.4s	remaining: 8.15s
2400:	learn: 19.6134483	total: 33s	remaining: 2.68s


[I 2023-11-11 12:01:14,678] Trial 0 finished with value: 52.789601417698265 and parameters: {'iterations': 2596, 'learning_rate': 0.002397452713005287, 'depth': 3, 'min_data_in_leaf': 51, 'l2_leaf_reg': 7, 'bagging_temperature': 0.6899130293316434, 'random_strength': 0.7725202236612505, 'border_count': 471, 'rsm': 0.9716122886346461}. Best is trial 0 with value: 52.789601417698265.


2595:	learn: 19.5124017	total: 35.7s	remaining: 0us
0:	learn: 63.1887240	total: 14.5ms	remaining: 50.4s
400:	learn: 19.8733037	total: 5.81s	remaining: 44.6s
800:	learn: 17.0989179	total: 11.8s	remaining: 39.5s
1200:	learn: 16.3957437	total: 17.9s	remaining: 33.9s
1600:	learn: 15.7034826	total: 24s	remaining: 28.1s
2000:	learn: 15.2442233	total: 30.1s	remaining: 22.2s
2400:	learn: 14.8267527	total: 36.4s	remaining: 16.3s
2800:	learn: 14.3937704	total: 42.7s	remaining: 10.3s
3200:	learn: 13.9539699	total: 49s	remaining: 4.21s


[I 2023-11-11 12:02:08,398] Trial 1 finished with value: 50.56964672849474 and parameters: {'iterations': 3476, 'learning_rate': 0.006938838681131012, 'depth': 7, 'min_data_in_leaf': 84, 'l2_leaf_reg': 4, 'bagging_temperature': 0.9569317095326304, 'random_strength': 0.40238628866924897, 'border_count': 75, 'rsm': 0.6101527019660291}. Best is trial 1 with value: 50.56964672849474.


...

0:	learn: 63.4388948	total: 61.4ms	remaining: 3m 5s
400:	learn: 41.8333698	total: 22.9s	remaining: 2m 29s
800:	learn: 29.2981802	total: 46.1s	remaining: 2m 8s
1200:	learn: 22.4446629	total: 1m 9s	remaining: 1m 45s
1600:	learn: 18.9156287	total: 1m 33s	remaining: 1m 23s
2000:	learn: 16.9925054	total: 2m 4s	remaining: 1m 3s
2400:	learn: 15.7584644	total: 2m 35s	remaining: 40.6s
2800:	learn: 14.9167192	total: 3m 5s	remaining: 15.2s
3029:	learn: 14.5742567	total: 3m 23s	remaining: 0us


[I 2023-11-11 12:31:54,699] Trial 19 finished with value: 49.508319915588096 and parameters: {'iterations': 3030, 'learning_rate': 0.001473824346819239, 'depth': 11, 'min_data_in_leaf': 1, 'l2_leaf_reg': 9, 'bagging_temperature': 0.6276313696625921, 'random_strength': 0.47123577918642, 'border_count': 421, 'rsm': 0.7106304922679701}. Best is trial 17 with value: 49.29546785784353.


0:	learn: 63.3443114	total: 105ms	remaining: 3m 31s
400:	learn: 25.7580357	total: 41.4s	remaining: 2m 46s
800:	learn: 17.3338037	total: 1m 22s	remaining: 2m 5s
1200:	learn: 14.8948492	total: 2m 4s	remaining: 1m 23s


[W 2023-11-11 12:34:11,921] Trial 20 failed with parameters: {'iterations': 2009, 'learning_rate': 0.0036061535848757147, 'depth': 11, 'min_data_in_leaf': 38, 'l2_leaf_reg': 10, 'bagging_temperature': 0.7801559655821352, 'random_strength': 0.6130387803097807, 'border_count': 612, 'rsm': 0.7614704494684774} because of the following error: KeyboardInterrupt('').
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/var/tmp/ipykernel_15194/4174548400.py", line 23, in <lambda>
    study.optimize(lambda trial: objective(trial, X_train, y_train), n_trials=80)
  File "/var/tmp/ipykernel_15194/4174548400.py", line 17, in objective
    catboost_model_c.fit(train_pool_c)
  File "/opt/conda/lib/python3.10/site-packages/catboost/core.py", line 5703, in fit
    return self._fit(X, y, cat_features, text_features, embedding_features, None, sample_weight, None, None, None, None, 

KeyboardInterrupt: 

In [7]:
#to output the best paramaters
print(study.best_params)

#to output the best score returned from the trials
print(study.best_value)

with open("optuna-best-parameters_c.txt", "w") as file:
    file.write("Best paramaters: \n")
    file.write(json.dumps(study.best_params))  # Write the first string followed by a newline character
    file.write("\n")
    file.write("best score MAE: \n")
    file.write(json.dumps(study.best_value))  # Write the second string followed by a newline character

{'iterations': 2004, 'learning_rate': 0.016429110886618033, 'depth': 8, 'colsample_bylevel': 0.7338575760364532, 'min_data_in_leaf': 90, 'l2_leaf_reg': 5, 'has-time': False, 'bagging_temperature': 0.6414768311294344, 'random_strength': 0.4206698081688342, 'border_count': 487, 'rsm': 0.012360675671015169, 'nan_mode': 'min'}
57.13529571086957
